In [ ]:
!pip install langchain openai chromadb sentence-transformers

# Scraping Feynman Lectures with metadata tags

In [2]:
from langchain.text_splitter import HTMLHeaderTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [5]:
def split_feynman(url):
  headers_to_split_on = [
    # ("h1", "Header 1"),
    ("h2", "Chapter"),
    ("h3", "Section")]

  html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

  # for local file use html_splitter.split_text_from_file(<path_to_file>)
  html_header_splits = html_splitter.split_text_from_url(url)

  chunk_size = 1000
  chunk_overlap = 0
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=chunk_size, chunk_overlap=chunk_overlap
  )

  # Split
  splits = text_splitter.split_documents(html_header_splits)
  return splits


In [6]:
base_url = "https://www.feynmanlectures.caltech.edu/"
vols = ['I', 'II', "III"]
chaps = [52, 42, 21]
# Loop through volumes and all chapters for each volume
all_chunks = []
for num, volume_num in enumerate(vols):
    chapters = chaps[num] +1
    for ch in range(1,chapters):
      vol_chapter_url = f"{base_url}{volume_num}_%02d.html" %ch
      splits = split_feynman(vol_chapter_url)
      for doc in splits:
        # Add volume metadata
        doc.metadata['Volume'] = volume_num
      all_chunks.extend(splits)

# Create Embeddings and Populate Vector Database

In [10]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma


In [ ]:

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

In [13]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db_hf'

## here we are using OpenAI embeddings but in future we will swap out to local embeddings
# embedding = OpenAIEmbeddings()

# embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")




vectordb = Chroma.from_documents(documents=all_chunks,
                                 embedding=hf,
                                 persist_directory=persist_directory)

In [49]:
# persist the db to disk
vectordb.persist()
vectordb = None

In [50]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

In [51]:
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 12})

In [52]:
query = "create a lesson plan about "
# metadata = {'source':'/content/PDFs/Nursing_Education different learning styles.pdf'}

In [53]:
# query = "kinematics"

docs_k = retriever.get_relevant_documents(query)#, metadata = metadata)

In [54]:
docs_k

[Document(page_content='By the following amount: if instead of letting the planet go around the sun, we were to change the direction (but not the magnitude) of its velocity and make it move radially, and then we let it fall from some special radius to the radius of interest, the new speed would be the same as the speed it had in the actual orbit, because this is just another example of a complicated path. So long as we come back to the same distance, the kinetic energy will be the same. So, whether the motion is the', metadata={'Chapter': '13Work and Potential Energy (A)', 'Section': '13–2Work done by gravity', 'Volume': 'I'}),
 Document(page_content='Now let us see how we can calculate the motion of Neptune, Jupiter, Uranus, or any other planet. If we have a great many planets, and let the sun move too, can we do the same thing? Of course we can. We calculate the force on a particular planet, let us say planet number\xa0$i$, which has a position $x_i,y_i,z_i$ ($i=1$ may represent the 

# popularity of top k by metadata

In [ ]:
# popularity of top k
# Create a dictionary to store the count of metadata occurrences
metadata_count = {}

# Iterate through the documents and count metadata occurrences
for document in docs_k:
    metadata = document.metadata
    for key, value in metadata.items():
        if key in metadata_count:
            if value in metadata_count[key]:
                metadata_count[key][value] += 1
            else:
                metadata_count[key][value] = 1
        else:
            metadata_count[key] = {value: 1}

# Print the count of metadata occurrences
for key, value_count in metadata_count.items():
    print(f"{key}:")
    for value, count in value_count.items():
        print(f"  {value}: {count} occurrences")

# semantic similarity to metadata

In [ ]:


# similarity with chapters/sections
# Semantic search over specific metadata tags (e.g. chapter, section)
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"filter":metadata,"k":5})
# Find ways to best use multiple sections, maybe cohere reranker
# Consider parent child retrieval

In [ ]:
metadata = {'Section': '18–1The center of mass'}
vectordb.get(where={'Section': '18–1The center of mass'})


#Experimenting on RAG vs Prompting for data within training set